In [8]:

from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Retrieve the API key
GROQ_API = os.getenv('GROQCLOUD_API_KEY')

if not GROQ_API:
    raise ValueError("GROQCLOUD_API_KEY is not set. Please check your .env file.")

# Initialize the ChatGroq object
llm = ChatGroq(
    api_key=GROQ_API,
    temperature=0,
    model_name='llama-3.1-70b-versatile'
)

# Use the model to generate a response
response = llm.invoke('The first person to land on the moon was ...')
print(response.content)


The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, as part of the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [16]:
from langchain_community.document_loaders import WebBaseLoader

# URL of the page to scrape
url = 'https://jobs.nike.com/job/R-46373?from=job%20search%20funnel'
# Initialize the web loader
loader = WebBaseLoader(url)

# Load the page content
page_data = loader.load().pop().page_content
print(page_data)


Apply for Director, Retail Marketing, South

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsMarketingDirector, Retail Marketing, SouthBarcelona, SpainBecome a Part of the NIKE, Inc. Tea

In [17]:
from langchain_core.prompts import PromptTemplate

prompt_extract=PromptTemplate.from_template(
    """### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### Instruction:
    the scraped text is from the career's page of website.
    your job is to extract the job posting and return them in JOSN format containing
    following keys:`role`,`exprience`,`skill` and `description`.
    only return the valid JSON.
    ## valid json (no problem)
    """
)

chain_extract=prompt_extract | llm
res=chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```json
{
  "role": "Director, Retail Marketing, South",
  "experience": "Extensive experience in Brand marketing, with a strong focus on retail marketing, and passion for brand storytelling excellence within our physical homes for the brand. Prior Nike Direct Marketing experience and operational excellence essential. Team leadership experience and experience of people management is essential for this role.",
  "skill": [
    "Brand marketing",
    "Retail marketing",
    "Brand storytelling",
    "Leadership",
    "People management",
    "English proficiency",
    "Italian or Spanish language skills (preferred)"
  ],
  "description": "We are looking for an experienced and talented brand marketeer with a passion for sport, a curiosity for youth and a desire to create a culture of creativity, innovation and inspiration. The Director, Retail Marketing, South will be part of the South marketing leadership team and lead the Retail Marketing function within the countries of Italy, Spain, P

In [18]:
from langchain_core.output_parsers import JsonOutputParser

json_parser=JsonOutputParser()
json_res=json_parser.parse(res.content)
json_res

{'role': 'Director, Retail Marketing, South',
 'experience': 'Extensive experience in Brand marketing, with a strong focus on retail marketing, and passion for brand storytelling excellence within our physical homes for the brand. Prior Nike Direct Marketing experience and operational excellence essential. Team leadership experience and experience of people management is essential for this role.',
 'skill': ['Brand marketing',
  'Retail marketing',
  'Brand storytelling',
  'Leadership',
  'People management',
  'English proficiency',
  'Italian or Spanish language skills (preferred)'],
 'description': 'We are looking for an experienced and talented brand marketeer with a passion for sport, a curiosity for youth and a desire to create a culture of creativity, innovation and inspiration. The Director, Retail Marketing, South will be part of the South marketing leadership team and lead the Retail Marketing function within the countries of Italy, Spain, Portugal & Greece.'}

In [20]:
import pandas as pd


In [21]:
df=pd.read_csv("../my_portfolio.csv")

In [23]:
import uuid  # Import the uuid module
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name='portfolio')

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row['Techstack'],
                       metadatas={'links': row['Links']},
                       ids=[str(uuid.uuid4())])